In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
dataset, info = tfds.load("tf_flowers",
                          as_supervised=True,
                          with_info=True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [ ]:
dataset_size = info.splits["train"].num_examples
class_names = info.features["label"].names
n_classes = info. features["label"].num_classes

In [ ]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load("tf_flowers", split=["train[:10%]",
                               "train[10%:25%]",
                               "train[25%:]"],
                        as_supervised=True)

In [ ]:
def preprocess(image,label):
    resized_image = tf.image.resize(image, [224,224])
    final_image = tf.keras.applications.xception.preprocess_input(resized_image)
    return final_image, label


In [ ]:
batch_size = 32
train_set = train_set_raw.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

In [ ]:
base_model = tf.keras.applications.xception.Xception(weights="imagenet",
                                            include_top=False)


avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)


model = tf.keras.Model(inputs=base_model.input, outputs=output)

83683744/83683744 [==============================] - 5s 0us/step


In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
optimizer = tf.keras.optimizers.SGD(lr=0.2,
                                    momentum=0.9,
                                    decay=0.01)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])


history = model.fit(train_set,
                    epochs=5,
                    validation_data=valid_set)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/5
86/86 [==============================] - 27s 159ms/step - loss: 2.1620 - accuracy: 0.7776 - val_loss: 1.4695 - val_accuracy: 0.8385
Epoch 2/5
86/86 [==============================] - 13s 145ms/step - loss: 0.5523 - accuracy: 0.9121 - val_loss: 0.9891 - val_accuracy: 0.8530
Epoch 3/5
86/86 [==============================] - 13s 146ms/step - loss: 0.2551 - accuracy: 0.9390 - val_loss: 0.8190 - val_accuracy: 0.8748
Epoch 4/5
86/86 [==============================] - 13s 147ms/step - loss: 0.1505 - accuracy: 0.9597 - val_loss: 0.8432 - val_accuracy: 0.8566
Epoch 5/5
86/86 [==============================] - 13s 147ms/step - loss: 0.1044 - accuracy: 0.9738 - val_loss: 0.8021 - val_accuracy: 0.8838


In [ ]:
for layer in base_model.layers:
  layer.trainable = True

In [ ]:
optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.01, momentum=0.9, decay=0.001)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

history = model.fit(train_set,
                    epochs=10,
                    validation_data=valid_set)

Epoch 1/10
86/86 [==============================] - 62s 509ms/step - loss: 0.5926 - accuracy: 0.7896 - val_loss: 0.3901 - val_accuracy: 0.8475
Epoch 2/10
86/86 [==============================] - 42s 487ms/step - loss: 0.1364 - accuracy: 0.9568 - val_loss: 0.2290 - val_accuracy: 0.9129
Epoch 3/10
86/86 [==============================] - 43s 496ms/step - loss: 0.0395 - accuracy: 0.9924 - val_loss: 0.2135 - val_accuracy: 0.9238
Epoch 4/10
86/86 [==============================] - 42s 491ms/step - loss: 0.0176 - accuracy: 0.9975 - val_loss: 0.2062 - val_accuracy: 0.9238
Epoch 5/10
86/86 [==============================] - 43s 495ms/step - loss: 0.0138 - accuracy: 0.9982 - val_loss: 0.2234 - val_accuracy: 0.9292
Epoch 6/10
86/86 [==============================] - 42s 491ms/step - loss: 0.0104 - accuracy: 0.9982 - val_loss: 0.2148 - val_accuracy: 0.9292
Epoch 7/10
86/86 [==============================] - 43s 495ms/step - loss: 0.0086 - accuracy: 0.9989 - val_loss: 0.2219 - val_accuracy: 0.9328